In [1]:
import iris

In [2]:
c = iris.load('./*.nc')

/opt/conda/lib/python3.5/site-packages/iris/fileformats/cf.py:1140: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


In [3]:
c

[<iris 'Cube' of wet_bulb_freezing_level_altitude / (m) (latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (time: 2; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_freezing_level_altitude / (m) (time: 2; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (time: 2; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of air_pressure_at_sea_level / (Pa) (time: 2; latitude: 600; longitude: 800)>,
<iris 'Cube' of air_pressure_at_sea_level / (Pa) (time: 2; latitude: 600; longitude: 800)>,
<iris 'Cube' of air_temperature / (K) (latitude: 600; longitude: 800)>,
<iris 'Cube' of air_temperature / (K) (latitude: 600; longitude: 800)>,
<iris 'Cube' of air_temperature / (K) (time: 2; pressure: 16; latitude: 600; longitude: 800)>,
<iris 'Cube' of air_temperature / (K) (latitude: 600; longitude: 800)>,
<iris 'Cube' of air_temperature / (K) (forecast_period: 2; latitude: 600; longitude: 800)>,
<iris '

In [4]:
sample = iris.cube.CubeList([c[16], c[17]])
sample

[<iris 'Cube' of fog_area_fraction / (1) (latitude: 600; longitude: 800)>,
<iris 'Cube' of fog_area_fraction / (1) (time: 2; latitude: 600; longitude: 800)>]

In [5]:
print(sample[0].coord('forecast_period'))
print(sample[1].coord('forecast_period'))

DimCoord(array([ 3.]), standard_name='forecast_period', units=Unit('hours'), var_name='forecast_period_0')
DimCoord(array([ 6.,  9.]), standard_name='forecast_period', units=Unit('hours'), var_name='forecast_period')


In [13]:
sample.concatenate_cube()

ConcatenateError: failed to concatenate into a single cube.
  Dimension coordinates differ: latitude, longitude != latitude, longitude, time
  Auxiliary coordinates differ: < None > != forecast_period
  Scalar coordinates differ: forecast_period, forecast_reference_time, time != forecast_reference_time
  Data dimensions differ: 2 != 3

In [57]:
s0 = sample[0]
print(s0)
print(s0.coord('time'))

fog_area_fraction / (1)             (latitude: 600; longitude: 800)
     Dimension coordinates:
          latitude                           x               -
          longitude                          -               x
     Scalar coordinates:
          forecast_period: 3.0 hours
          forecast_reference_time: 2016-01-01 00:00:00
          time: 2016-01-01 03:00:00
     Attributes:
          Conventions: CF-1.5
          STASH: m01s03i248
          source: Data from Met Office Unified Model
          um_version: 10.1
DimCoord([2016-01-01 03:00:00], standard_name='time', calendar='gregorian', var_name='time_0')


In [33]:
s1 = sample[1]
print(s1)
t_coord = s1.coord('time')

fog_area_fraction / (1)             (time: 2; latitude: 600; longitude: 800)
     Dimension coordinates:
          time                           x            -               -
          latitude                       -            x               -
          longitude                      -            -               x
     Auxiliary coordinates:
          forecast_period                x            -               -
     Scalar coordinates:
          forecast_reference_time: 2016-01-01 00:00:00
     Attributes:
          Conventions: CF-1.5
          STASH: m01s03i248
          source: Data from Met Office Unified Model
          um_version: 10.1


In [37]:
t_coord.ndim

1

In [39]:
from iris.util import promote_aux_coord_to_dim_coord, new_axis, demote_dim_coord_to_aux_coord
c = new_axis(s0, 'time')
print(c)


fog_area_fraction / (1)             (time: 1; latitude: 600; longitude: 800)
     Dimension coordinates:
          time                           x            -               -
          latitude                       -            x               -
          longitude                      -            -               x
     Scalar coordinates:
          forecast_period: 3.0 hours
          forecast_reference_time: 2016-01-01 00:00:00
     Attributes:
          Conventions: CF-1.5
          STASH: m01s03i248
          source: Data from Met Office Unified Model
          um_version: 10.1


In [70]:
scalar_coord = c.coord('forecast_period')
aux_coord = iris.coords.DimCoord.from_coord(scalar_coord)
aux_coord

DimCoord(array([ 3.]), standard_name='forecast_period', units=Unit('hours'), var_name='forecast_period')

In [72]:
c.remove_coord('forecast_period')
c.add_aux_coord(aux_coord, 0)

In [73]:
c

<iris 'Cube' of fog_area_fraction / (1) (time: 1; latitude: 600; longitude: 800)>

In [74]:
print(c)

fog_area_fraction / (1)             (time: 1; latitude: 600; longitude: 800)
     Dimension coordinates:
          time                           x            -               -
          latitude                       -            x               -
          longitude                      -            -               x
     Auxiliary coordinates:
          forecast_period                x            -               -
     Scalar coordinates:
          forecast_reference_time: 2016-01-01 00:00:00
     Attributes:
          Conventions: CF-1.5
          STASH: m01s03i248
          source: Data from Met Office Unified Model
          um_version: 10.1


In [75]:
print(s1)

fog_area_fraction / (1)             (time: 2; latitude: 600; longitude: 800)
     Dimension coordinates:
          time                           x            -               -
          latitude                       -            x               -
          longitude                      -            -               x
     Auxiliary coordinates:
          forecast_period                x            -               -
     Scalar coordinates:
          forecast_reference_time: 2016-01-01 00:00:00
     Attributes:
          Conventions: CF-1.5
          STASH: m01s03i248
          source: Data from Met Office Unified Model
          um_version: 10.1


In [49]:
cl = iris.cube.CubeList([c, s1])
cl

[<iris 'Cube' of fog_area_fraction / (1) (time: 1; latitude: 600; longitude: 800)>,
<iris 'Cube' of fog_area_fraction / (1) (time: 2; latitude: 600; longitude: 800)>]

In [52]:
cl.concatenate_cube()

ConcatenateError: failed to concatenate into a single cube.
  Dimension coordinates metadata differ: time != time
  Auxiliary coordinates metadata differ: forecast_period != forecast_period

In [76]:
c.coord('time')

DimCoord(array([ 403227.]), standard_name='time', units=Unit('hours since 1970-01-01 00:00:00', calendar='gregorian'), var_name='time')

In [77]:
s1.coord('time')

DimCoord(array([ 403230.,  403233.]), standard_name='time', units=Unit('hours since 1970-01-01 00:00:00', calendar='gregorian'), var_name='time')

In [78]:
c.coord('time').var_name='time'
c.coord('time')

DimCoord(array([ 403227.]), standard_name='time', units=Unit('hours since 1970-01-01 00:00:00', calendar='gregorian'), var_name='time')

In [79]:
c.coord('forecast_period')

DimCoord(array([ 3.]), standard_name='forecast_period', units=Unit('hours'), var_name='forecast_period')

In [80]:
s1.coord('forecast_period')

DimCoord(array([ 6.,  9.]), standard_name='forecast_period', units=Unit('hours'), var_name='forecast_period')

In [81]:
c.coord('forecast_period').var_name = 'forecast_period'

In [82]:
c.has_lazy_data()

False

In [83]:
s1.has_lazy_data()

False

In [84]:
cl.concatenate_cube()

<iris 'Cube' of fog_area_fraction / (1) (time: 3; latitude: 600; longitude: 800)>